In [34]:
from datasets import load_dataset

In [35]:
qa2d = load_dataset("domenicrosati/QA2D", split="train[:50000]")

In [38]:
qa2d = qa2d.train_test_split(test_size=0.1)

In [39]:
qa2d["train"][0]

{'dataset': 'SQuAD',
 'example_uid': '57314737497a881900248d3c',
 'question': 'What is the name of a type of dual purpose fighter-bomber aircraft used by the US Air Force ?',
 'answer': 'F-16 Fighting Falcon',
 'turker_answer': 'The F-16 Fighting Falcon is the name of a type of dual purpose fighter-bomber aircraft used by the US Air Force .',
 'rule-based': 'The name of a type of dual purpose fighter-bomber aircraft used by the US Air Force is F-16 Fighting Falcon .'}

In [40]:
def filter_dataset(example):
    """
    Make sure that the samples used for training
    1. Have an answer
    2. Have a question 
    3. Has an answer (turker_answer)
    """
    usable = example["answer"] != "" and example["question"] != "" and example["turker_answer"] != ""
    return usable

In [41]:
qa2d.filter(filter_dataset)

Filter:   0%|          | 0/45000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['dataset', 'example_uid', 'question', 'answer', 'turker_answer', 'rule-based'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['dataset', 'example_uid', 'question', 'answer', 'turker_answer', 'rule-based'],
        num_rows: 5000
    })
})

In [44]:
checkpoint = "google-t5/t5-small"

In [45]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [50]:
def generate_targets(examples):
    # The output is of format "question? answer"
    return list(map(
        lambda q, a: f"{q} {a}",
        examples["question"],
        examples["answer"],
    ))

In [51]:
generate_targets(qa2d["train"][:3])

['What is the name of a type of dual purpose fighter-bomber aircraft used by the US Air Force ? F-16 Fighting Falcon',
 'What month did Bell go to Boston ? April',
 'What was the driving force behind the revitalization of the Roman naval forces ? to meet several new demands']

In [52]:
def preprocess_function(examples):
    """ 
    The objective of our model is to transform a sentence into a question.
    To do so, the input to the model will be the sentence itself.
    The output of the model is be the question followed by the answer.
    """
    prompt = "ask: " # Each input will be formatted as "ask: sentence..."
    inputs = [prompt + x for x in examples["turker_answer"]] 
    # The output is of format "question? answer"
    targets = generate_targets(examples)

    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [61]:
statements = qa2d["train"][20:24]
for sample in zip(statements["turker_answer"], generate_targets(statements)):
    print(f"{sample[0]} => {sample[1]}")

Jack Brickhouse manned the Cubs radio and TV booth for parts of five decades . => Who manned the Cubs radio and TV booth for parts of five decades ? Jack Brickhouse
The refractive index of air is n = 1 . => What is the refractive index of air ? n = 1
27 % of European digital satellite TV homes were watching HD broadcast in 2010 . => What percentage of European digital satellite TV homes were watching HD broadcasts in 2010 ? 27 %
They have an opportunity to appeal charges . => What did they have an opportunity to appeal ? charges


In [62]:
tokenized_qa2d = qa2d.map(preprocess_function, batched=True)

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [63]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [1]:
""""
Popular text2text generation tasks are machine translation, 
commonly evaluated with the BLEU score and a focus on word precision, 
and text summarization, commonly evaluated with the ROUGE score and a focus on word recall.
"""

import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

D:\MyProjects\DawsonAI\.env\Lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [65]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [67]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_qa2d",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=4,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=10,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_qa2d["train"],
    eval_dataset=tokenized_qa2d["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

D:\MyProjects\DawsonAI\.env\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [68]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.578900,0.557558,0.829700,0.665500,0.744600,0.744500,17.137200
2,0.585900,0.524811,0.840300,0.688100,0.761200,0.761000,17.113800
3,0.634500,0.512762,0.842800,0.694800,0.767200,0.767000,17.115600
4,0.578500,0.509624,0.843500,0.696400,0.767900,0.767700,17.118800


D:\MyProjects\DawsonAI\.env\Lib\site-packages\transformers\generation\utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
D:\MyProjects\DawsonAI\.env\Lib\site-packages\transformers\generation\utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
D:\MyProjects\DawsonAI\.env\Lib\site-packages\transformers\generation\utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
D:\MyProjects\DawsonAI\.env\Lib\site-packages\transformers\generation\utils.py:1178: UserWarning: Using the model-agnostic default `max_

TrainOutput(global_step=11252, training_loss=0.6435050720046349, metrics={'train_runtime': 2926.2111, 'train_samples_per_second': 61.513, 'train_steps_per_second': 3.845, 'total_flos': 1746638815887360.0, 'train_loss': 0.6435050720046349, 'epoch': 4.0})

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

q_model = AutoModelForSeq2SeqLM.from_pretrained("results_qa2d_t5_small/checkpoint-11252")
tokenizer = AutoTokenizer.from_pretrained("results_qa2d_t5_small/checkpoint-11252")

In [3]:
from nltk.tokenize import sent_tokenize

In [7]:
# Don't forget to add the "ask: " prompt!
input_text = "Today it is Dante's birthday."
# Use NLTK to tokenize the whole text into sentences
sentences = sent_tokenize(input_text)
for sentence in sentences:
    # Don't forget to add our prefix for prompting!
    text_input = "ask: " + sentence
    input_ids = tokenizer(text_input, return_tensors="pt").input_ids
    outputs = q_model.generate(input_ids, max_new_tokens=100, do_sample=False)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

What is Dante's birthday? Today
